In [1]:
from kedro.runner.sequential_runner import SequentialRunner
from kedro.io.data_catalog import DataCatalog
from jaffle_shop.utilities.graph import ibis_expr_to_graph,kedro_pipeline_to_graph
from jaffle_shop.utilities.context import all_logging_disabled

In [3]:
default_pipeline = pipelines['__default__']
default_pipeline.datasets()


{
    'orders',
    'stg_customers',
    'temp_orders',
    'params:payment_methods',
    'customers',
    'raw_customers',
    'stg_orders',
    'raw_payments',
    'stg_payments',
    'raw_orders'
}

In [4]:
default_pipeline = pipelines['__default__']

with all_logging_disabled():
    free_outputs = SequentialRunner().run(
        pipeline=default_pipeline.filter(
            tags=None,
            from_nodes = None,
            to_nodes = None,
            node_names = None,
            from_inputs = None,
            to_outputs = None,
            node_namespace = None,
        ),
        catalog=DataCatalog(
            feed_dict={
                x: catalog.load(x).head(0)
                if 'params:' not in x else catalog.load(x)
                for x in default_pipeline.inputs()

            }
        ),
     #   hook_manager=
    )

        

In [33]:

nx_ibis_graphs = {k:ibis_expr_to_graph(v) for k,v in free_outputs.items()}
nx_ibis_graphs


{
    'orders': <networkx.classes.digraph.DiGraph object at 0x1647c1600>,
    'temp_orders': <networkx.classes.digraph.DiGraph object at 0x1647c2770>,
    'customers': <networkx.classes.digraph.DiGraph object at 0x1647c07c0>
}

In [7]:
nx_kedro_graph = kedro_pipeline_to_graph(default_pipeline, catalog)

In [77]:
from jaffle_shop.utilities.graph import render_nx_graph, render_kedro_node

render_nx_graph(nx_kedro_graph, color_field="attr_type", title_attr="obj").show('graph.html')

graph.html
graph.html


<IPython.lib.display.IFrame object at 0x16899bbb0>

In [35]:
render_nx_graph(
    nx_ibis_graphs['orders'], 
    color_field="operation_name", 
    label_attr="operation_name",
    title_attr="data_name"
).show('graph.html')

graph.html
graph.html


<IPython.lib.display.IFrame object at 0x1647d7190>

In [43]:
n = default_pipeline.nodes[4]
render_kedro_node(n,catalog).show('graph.html')

[02/22/24 16:48:23] INFO     Loading data from stg_orders (TableDataset)...                     ]8;id=514552;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=390428;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Loading data from stg_payments (TableDataset)...                   ]8;id=356716;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=696030;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Loading data from params:payment_methods (MemoryDataset)...        ]8;id=106848;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=394387;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Running node:                                                              ]8;id=539915;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=940770;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/pipeline/node.py#340\340]8;;\
                             process_orders([stg_orders;stg_payments;params:payment_methods]) ->                   
                             [orders;temp_orders]                                                                  

graph.html
graph.html


<IPython.lib.display.IFrame object at 0x164203250>

In [87]:
import networkx as nx
from itertools import chain

node_outputs = n.run(
    {
        x: catalog.load(x).head(0) if "params:" not in x else catalog.load(x)
        for x in n.inputs
    }
)

ibis_output_graphs = {output : ibis_expr_to_graph(g) for output, g in node_outputs.items()}

last_topo_sorts = {output: list(nx.topological_generations(g))[-1] for output, g in ibis_output_graphs.items()}


last_topo_sorts

#nx_ibis_graphs = nx.compose_all(ibis_graphs)





[02/22/24 17:09:03] INFO     Loading data from stg_orders (TableDataset)...                     ]8;id=886475;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=51456;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Loading data from stg_payments (TableDataset)...                   ]8;id=588809;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=764338;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Loading data from params:payment_methods (MemoryDataset)...        ]8;id=893774;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=81660;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

                    INFO     Running node:                                                              ]8;id=603947;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=911768;file:///Users/joel_schwarzmann/anaconda3/envs/jaffle-lineage/lib/python3.10/site-packages/kedro/pipeline/node.py#340\340]8;;\
                             process_orders([stg_orders;stg_payments;params:payment_methods]) ->                   
                             [orders;temp_orders]                                                                  

{'orders': ['2435726319686336928'], 'temp_orders': ['-6434569716298973142']}

In [89]:
ibis_output_graphs['temp_orders'].nodes['-6434569716298973142']


{
    'data_name': None,
    'operation_name': 'Sample',
    'types': {
        'order_id': Int64(nullable=True),
        'customer_id': Int64(nullable=True),
        'order_date': Date(nullable=True),
        'status': String(nullable=True)
    }
}

In [82]:
node_outputs.keys()

dict_keys(['orders', 'temp_orders'])